In [1]:
# install.packages("devtools")
# install.packages("nat")
# install.packages("apcluster")
# install.packages("dendextend")
# install.packages("dynamicTreeCut")
# install.packages("gplots")
# install.packages("RColorBrewer")
# install.packages("d3heatmap")
# install.packages("Rtsne")
# install.packages("jsonlite")
# install.packages("sommer")
# install.packages("FreeSortR")
# install.packages("hdf5r")
# install.packages("doMC")

library(doMC)
registerDoMC()

DATASET_DIR = "../01_data/"

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



# NBLAST clustering

In [8]:
library(jsonlite)
library(dendextend)
library(hdf5r)

f <- H5File$new(file.path(DATASET_DIR, "atlas/nblast/data.h5"), mode="r")
f$ls(recursive=TRUE)

neuronIdList_ds <- f[["neuronIdList"]]

# typeof(neuronIdList_ds)
neuronIdList <- neuronIdList_ds[]
similarityMatrix <- f[["meanScoreMatrix"]]$read()

f$close_all()

neuronIdList %>% length

,name,link.type,obj_type,num_attrs,group.nlinks,group.mounted,dataset.rank,dataset.dims,dataset.maxdims,dataset.type_class,dataset.space_class,committed_type
,<chr>,<factr_xt>,<factr_xt>,<int>,<int>,<int>,<int>,<chr>,<chr>,<factr_xt>,<factr_xt>,<factr_xt>
1,meanScoreMatrix,0,5,0,NA,NA,2,3037 x 3037,3037 x 3037,1,1,NA
2,neuronIdList,0,5,0,NA,NA,1,3037,3037,0,1,NA
3,scoreMatrix,0,5,0,NA,NA,2,3037 x 3037,3037 x 3037,1,1,NA


[1] 3037

In [18]:
###### library("dendextend")
library("dynamicTreeCut")
distanceMatrix = -similarityMatrix + max(similarityMatrix)



dist = as.dist(distanceMatrix)
hc = hclust(dist, method="ward.D")

k = 31
dend <- hc %>% as.dendrogram #%>% 
            #color_branches(k=k, col=rainbow_hcl(k), groupLabels=TRUE) # %>% set("labels", NULL)
clusters = cutree(hc, k=k)

# clusters <- cutreeDynamic(hc, distM=distanceMatrix, method="tree")


orders = order.hclust(hc)
orderedClusters = clusters[orders]
orderedNeuronIdList = neuronIdList[orders]


neuronGroups = c()
i = 0
groupIdList <- c()
for (groupId in unique(orderedClusters)){
    neuronGroup = orderedNeuronIdList[orderedClusters==groupId]
    neuronGroupList <- c(neuronGroupList, list(neuronGroup))
    groupIdList <- c(groupIdList, groupId)
    
    cat("id: ", i, ": ", neuronGroup, "\n\n")
    i = i + 1
}

# neuronGroupList

# write as json file
library(jsonlite)
write(toJSON(neuronGroupList), file=file.path(DATASET_DIR, "atlas/nblast/neuron_groups.json"))

id:  0 :  82882 76607 83169 82771 82885 82758 77898 76454 80279 82812 76555 76556 82909 82917 82898 81556 77249 76544 82976 77450 82807 77897 78065 82770 82824 82914 82945 83188 

id:  1 :  82722 82726 82723 82725 82719 82721 82718 82720 76833 78554 81548 81620 82592 78627 79047 79827 78439 79032 82929 76904 76908 81001 76568 77911 79197 83185 80740 77654 77302 78440 83186 82140 78581 81526 78582 81185 82145 82143 82146 82196 82197 77295 82212 77646 82852 77018 79401 82623 83105 78684 78459 79045 76677 79452 80742 76872 77059 82757 77105 76921 76463 77086 77030 77090 83314 77887 77893 77044 76465 76491 76492 78852 79533 83338 76466 77152 82174 83171 83343 78250 82713 82264 82714 77940 83335 77941 76877 77153 77135 77247 78567 76402 76455 83182 79083 77157 82710 78550 77901 79053 83183 77231 77942 77125 82709 77128 76496 76497 76537 77939 77938 76530 77928 82777 77667 83243 79580 77109 83248 83284 77148 79051 82213 82717 81145 77150 78547 79084 77137 78685 77250 78557 77299 80803 83178 

# Manually split neuron group

In [4]:
# library(jsonlite)
# file_path = file.path(DATASET_DIR, "atlas/nblast/manual_split_neuron_groups.json")
# neuronGroupList = fromJSON(file_path)

In [53]:
# splitGroupId = 0
# cat("split group ", splitGroupId, "\n\n")

# splitNeuronGroup = neuronGroupList[[ splitGroupId+1 ]]

# groupStr = "82776, 82778, 82783, 82773, 82775, 82782, 79691, 80273, 80274, 77828, 80207, 77873, 79131, 80165, 76778, 80544, 80570, 76354, 78400, 78410, 79196, 79699, 77063, 76657, 77610, 77578, 77598, 76392, 78409, 78450, 76263, 80604, 80748, 81757, 79859, 78701, 82646, 81074, 77802, 78294, 81120, 82178, 82185, 82186, 82173, 82170, 82172, 82184, 82189, 82179, 82187, 83166, 77112, 82175, 77110, 82181, 78057, 82180, 82176, 82188, 83167, 83175, 83176, 82190, 83172, 78058, 78059, 82182, 78060, 78682, 76410, 76415, 77579, 83353, 76409, 83357, 76405, 80603, 77348, 80187, 78603, 81076, 81747, 77857, 78595, 76190, 78335, 76270, 76292, 80224, 83354, 76193, 76198, 82905, 76875, 82171, 82938, 76393, 76462, 78419, 82910, 76195, 78452, 79361, 77778, 78615"
# groupStr = "82776, 82778, 82783, 82773, 82775, 82782, 79691, 80273, 80274, 77828, 80207, 77873, 79131, 80165, 76778, 80544, 80570, 76354, 78400, 78410, 79196, 79699, 77063, 76657, 77610, 77578, 77598, 76392, 78409, 78450, 76263, 80604, 80748, 81757, 79859, 78701, 82646, 81074, 77802, 78294, 81120"
# groupStr = "82178, 82185, 82186, 82173, 82170, 82172, 82184, 82189, 82179, 82187, 83166, 77112, 82175, 77110, 82181, 78057, 82180, 82176, 82188, 83167, 83175, 83176, 82190, 83172, 78058, 78059, 82182, 78060, 78682, 76410, 76415, 77579, 83353, 76409, 83357, 76405, 80603, 77348, 80187, 78603, 81076, 81747, 77857, 78595, 76190, 78335, 76270, 76292, 80224, 83354, 76193, 76198, 82905, 76875, 82171, 82938, 76393, 76462, 78419, 82910, 76195, 78452, 79361, 77778, 78615"
# groupStr = "80274, 80273, 80165, 80207, 77828, 76778, 77873, 80570, 76354, 78400, 78410, 76657, 77610, 77578, 77598, 76392, 78409, 78450, 76263, 80604, 81074, 77802, 78294, 81120, 76410, 76415, 77579, 83353, 76409, 83357, 76405, 80603, 77348, 80187, 78603, 81076, 81747, 77857, 78595, 76190, 78335, 76270, 76292, 80224, 83354, 76193, 76198, 82905, 82938, 76393, 78419, 82910, 76195, 78452, 82866, 82930, 82939, 78233, 76264, 76348, 78235, 78333, 82933, 82954, 82919, 77244, 82879, 82912, 82963, 82876, 82940, 78762, 78442, 76403, 78397, 76867, 80573, 76289, 76474, 76349, 80612, 76268, 76894, 76897, 76893, 77597, 80954, 78437, 77069, 78413, 77075, 79018, 78393, 78399, 77078, 82765, 77066, 78454, 78764, 78771, 79008, 78430, 78460, 76635, 78457, 79036, 78405, 83080, 77076, 81013, 82398, 83068, 78239, 80355, 78599, 80275, 78308, 82600, 80808, 76868, 76869, 76401, 76864, 76858, 79043, 77061, 78392, 81505, 82533, 76633, 78232, 81062, 82262, 79009, 77589, 76407, 76473, 82911, 83351, 76471, 77825, 79024, 80313, 78831, 79025, 76404, 76406, 79137, 82446, 81513, 78391, 76293, 77072, 79142, 82164, 78164, 81025, 78775, 79820, 77233, 76879, 77934, 77472, 78829, 82683, 78888, 80536, 76353, 77523, 76863, 78063, 76624, 76878, 78434, 77595, 76880, 77599, 82678, 79046, 77588, 82887, 77068, 77594, 77611, 77060, 79017, 77085, 77088, 77089, 77070, 78435, 83270, 79040, 76383, 76634, 78447, 78448"
groupStr = "80352, 78274, 78293, 80527, 78231, 80517, 78258, 80326, 78317, 80349, 80325, 80348, 80237, 77751, 80256, 78313, 80493, 82367, 80240, 78264, 80360, 80347, 80346, 80350, 79548, 79765, 80610, 79128, 78766, 79171, 79647, 79829, 77824, 82360, 79838, 79969, 81750, 82649, 82456, 79978, 78628, 79239, 83120, 81795, 79200, 80936, 79719, 79830, 78836, 80841, 79694, 81751, 81850, 79441, 79759, 79422, 81657, 83142, 79861, 81101, 78819, 81089, 78162, 79968, 79979, 78173, 78830, 79704, 78608, 81844, 78142, 79220, 78779, 79817, 83081, 78136, 78854, 81299, 76957, 78118, 79158, 82651, 83123, 82590, 79167, 78261, 80554, 78266, 80303, 83065, 83085, 78230, 78259, 82875, 77248, 78820, 83066, 78181, 81831, 82576, 79254, 78789, 79134, 78129, 78148, 78100, 78301, 79516, 80503, 78229, 78268, 80353, 78262, 78318, 80357, 78260, 78267, 78320, 78269, 78273, 78328, 80291, 78245, 78282, 78236, 78237, 80323, 80337, 80338, 78253, 80345, 80341, 77251, 80339, 78270, 78307"
splitNeuronGroup = as.integer(strsplit(groupStr, ",")[[1]])

for (k in 30:2000){
    clusters = cutree(hc, k=k)

    ordersTmp = order.hclust(hc)
    orderedClustersTmp = clusters[ordersTmp]
    orderedNeuronIdListTmp = neuronIdList[ordersTmp]

    findSplit = FALSE
    for (groupId in unique(orderedClustersTmp)){
        neuronGroup = orderedNeuronIdListTmp[orderedClustersTmp==groupId]
        if (length(neuronGroup)<length(splitNeuronGroup) &  any(neuronGroup %in% splitNeuronGroup)){
            findSplit = TRUE
            cat(neuronGroup, "\n\n")
        }
    }
    if (findSplit){
        break
    }
}


80352 78274 78293 80527 78231 80517 78258 80326 78317 80349 80325 80348 80237 77751 80256 78313 80493 82367 80240 78264 80360 80347 80346 80350 79548 79765 80610 79128 78766 79171 79647 79829 77824 82360 79838 79969 81750 82649 82456 79978 78628 79239 83120 81795 79200 80936 79719 79830 78836 80841 79694 81751 81850 79441 79759 79422 81657 83142 79861 81101 78819 81089 78162 79968 79979 78173 78830 79704 78608 81844 78142 79220 78779 79817 

83081 78136 78854 81299 76957 78118 79158 82651 83123 82590 79167 78261 80554 78266 80303 83065 83085 78230 78259 82875 77248 78820 83066 78181 81831 82576 79254 78789 79134 78129 78148 78100 78301 79516 80503 78229 78268 80353 78262 78318 80357 78260 78267 78320 78269 78273 78328 80291 78245 78282 78236 78237 80323 80337 80338 78253 80345 80341 77251 80339 78270 78307 



# Find the cut tree threshold

In [ ]:
library(dendextendRcpp)
hpk = dendextendRcpp_heights_per_k.dendrogram(dend)
cutree_height = hpk[k]
cat("cutting height of dendrogram: ", cutree_height)

# library(colorspace)
library(gplots)
library(RColorBrewer)

# png(file=file.path(DATASET_DIR, "figs/nblast/dend.png"))
# svg(file=file.path(DATASET_DIR, "figs/nblast/dend.svg"))
group_colors = colorRampPalette(brewer.pal(12, "Set3"))(length(unique(clusters)))
dend <- color_branches(dend, k=k, col=group_colors[groupIdList], 
                       groupLabels=FALSE) %>% set("labels", NULL) %>% 
        highlight_branches_lwd(values = seq(1, 4, length.out = 1000))
plot(dend)
abline(h=cutree_height, lty=2)

# dev.off()

In [ ]:
library(ggplot2)

ggplot(as.ggdend(dend %>% highlight_branches_lwd(values = seq(0.2, 2, length.out = 1000))))


In [ ]:
# # let's get the clusters
# library(dynamicTreeCut)
# data(iris)
# x  <- iris[,-5] %>% as.matrix
# hc <- x %>% dist %>% hclust
# dend <- hc %>% as.dendrogram 

# # Find special clusters:
# clusters <- cutreeDynamic(hc, distM = as.matrix(dist(x)), method = "tree")
# # we need to sort them to the order of the dendrogram:
# clusters <- clusters[order.dendrogram(dend)]
# clusters_numbers <- unique(clusters) - (0 %in% clusters)
# n_clusters <- length(clusters_numbers)

# library(colorspace)
# cols <- rainbow_hcl(n_clusters)
# true_species_cols <- rainbow_hcl(3)[as.numeric(iris[,][order.dendrogram(dend),5])]
# dend2 <- dend %>% 
#          branches_attr_by_clusters(clusters, values = cols) %>% 
#          color_labels(col =   true_species_cols)
# plot(dend2)
# clusters <- factor(clusters)
# levels(clusters)[-1]  <- cols[-5][c(1,4,2,3)] 
#    # Get the clusters to have proper colors.
#    # fix the order of the colors to match the branches.
# colored_bars(clusters, dend, sort_by_labels_order = FALSE)

In [ ]:
i = 0
groupIdList <- c()
for (groupId in unique(orderedClusters)){
    neuronGroup = orderedNeuronIdList[orderedClusters==groupId]
    neuronGroupList <- c(neuronGroupList, list(neuronGroup))
    groupIdList <- c(groupIdList, groupId)
    i = i + 1
    cat("id: ", i, "group, ", groupId, "\n\n")
}


In [ ]:
# # save orders for further analysis
# f = h5file(file.path(DATASET_DIR, "data.h5"))
# f["nblast/zfishTable/orders"] = orders
# f["nblast/zfishTable/clusters"] = clusters
# f["nblast/zfishTable/orderedClusters"] = orderedClusters
# f["nblast/zfishTable/orderedNeuronIdList"] = orderedNeuronIdList
# h5close(f)

# Plot the dendrogram with distance matrix

In [ ]:
library(gplots)

# library(heatmap3)


# svg(filename=file.path(DATASET_DIR, "fig/hclust.svg"))
png(filename=file.path(DATASET_DIR, "fig/hclust.png"), width=480*4, height=480*4, res=300)
rc <- group_colors[clusters]
cc <- rc

hv <- heatmap.2(distanceMatrix, 
# hv <- heatmap3(distanceMatrix, 
                distfun=as.dist, 
                hclustfun = function(x) hclust(x, method="ward.D"),
                Rowv = dend, Colv = dend, 
                col = brewer.pal(9, "YlGnBu"), 
                RowSideColors = rc, ColSideColors = cc,
                trace = "none",
                labRow = NA, labCol = NA,
                useRaster=TRUE)

dev.off()

In [ ]:
# library(ggplot2)
# ggsave(file.path(DATASET_DIR, "figs/nblast/hclust.svg"), device=svg())

# Order connectivity matrix

In [ ]:
connMatrix[1,]

In [ ]:
# read connectivity matrix
# f = h5file(file.path(DATASET_DIR, "conn.h5"))
# connMatrix = f["connMatrix"][]
# h5close(f)

orderedPreSynIdxes = orders[connMatrix[1,]]
orderedPostSynIdxes = orders[connMatrix[2,]]
synNums = connMatrix[3,]
psdSizes = connMatrix[4,]

data = data.frame("orderedPreSynIdxes"=orderedPreSynIdxes, 
                  "orderedPostSynIdxes"=orderedPostSynIdxes, 
                  "synNums"=synNums, 
                  "psdSizes"=psdSizes)


assertthat::assert_that(max(psdSizes)>1000)

In [ ]:
synNums[orders] %>% length

In [ ]:
# library(plotly)

# p <- plot_ly(data = data, x = ~orderedPreSynIdxes, y = ~orderedPostSynIdxes,
#         marker = list(size = 10,
#                        color = 'rgba(255, 182, 193, .9)',
#                        line = list(width = 0))) %>%
#   layout(title = 'connectivity matrix',
#          yaxis = list(title = 'postsynapse'),
#          xaxis = list(zeroline = 'presynapse'))

# # Create a shareable link to your chart
# # Set up API credentials: https://plot.ly/r/getting-started
# chart_link = api_create(p, filename="scatter-styled")
# chart_link

In [ ]:
# # pdf(file="~/Downloads/hclust.pdf")
# library(sommer)
# X = connMatrix[1,][orders]
# Y = connMatrix[2,][orders]
# synNum = connMatrix[3,][orders]
# psdSize = connMatrix[4,][orders]

library(ggplot2)

# # col = jet.colors(max(synNum))[synNum]
# # qplot(x=X, y=Y, alpha=0.1, size=synNum, stroke=F, 
# #       xlab="presynapse", ylab="postsynapse",
# #      asp=1) + xlim(0,NA) + ylim(0,NA)

# # qplot(x=X, y=Y, alpha=0.1, size=I(1), color=col, stroke=F, 
# #       xlab="presynapse", ylab="postsynapse",
# #      asp=1) 

ggplot(data, aes(orderedPreSynIdxes, orderedPostSynIdxes,  colour = synNums)) + 
  geom_point(alpha=0.2, size=1, stroke=0) + xlab("presynapse") + ylab("postsynapse") #+ coord_cartesian(xlim = c(0,length(X)), ylim = (0,length(Y)))

# ggsave(file.path(DATASET_DIR, "figs/synapse_num_connectivity_matrix_ordered_by_morphological_clustering.svg"))

In [ ]:
coord_cartesian(xlim = c(0,length(X)), ylim = (0,length(Y)))

In [ ]:
# d3heatmap will crash the browser!

# library(d3heatmap)
# d3heatmap(distanceMatrix, 
#           distfun = as.dist, 
#           hclustfun = function(x) hclust(x, method="ward.D"),
#           Rowv = dend, Colv = dend)

# R NBLAST

In [ ]:
# library(nat)
# library(nat.nblast)

# fileNameList = paste0(file.path(DATASET_DIR, "swc/"), neuronIdList, ".swc")

# neurons = read.neurons(fileNameList, neuronnames=neuronIdList)
# # divide by 1000 to transform unit from nm to micron
# mydotprops = dotprops(neurons/1e3, resmaple=1, k=10)

# # plot3d(mydotprops)

# # aba=nblast_allbyall(mydotprops, version=1, sd=3000)

# start_time <- Sys.time()
# aba = nblast_allbyall(mydotprops, normalisation='mean')
# end_time <- Sys.time()
# end_time - start_time
# heatmap(aba)

In [ ]:
# start_time <- Sys.time()
# aba = nblast_allbyall(mydotprops, normalisation='mean')
# end_time <- Sys.time()
# end_time - start_time
# aba

In [ ]:
# aba = sub_score_mat(scoremat = aba, normalisation = 'normalised')
# aba

In [ ]:
# aba = sub_score_mat(scoremat = aba, normalisation = 'mean')

In [ ]:
# hc=nhclust(scoremat=aba)
# pdf(file="~/Downloads/nhclust.pdf", width=46)
# plot(hc)
# dev.off()
# # plot3d(hc, k=3, db=mynlist)

In [ ]:
# hc2=nhclust(scoremat=aba, method="ward.D2")
# plot(hc2)

In [ ]:
library(dendroextras)
d=colour_clusters(hc, k=4)
# change dendrogram labels to neuron type, extracting this information
# from type column in the metadata data.frame attached to kcs20 neuronlist
# labels(d)=with(kcs20[labels(dkcs)], type)
plot(d)

In [ ]:
write.csv(aba, file=paste(DATA_DIR, "nblast_", listName, "_distance_matrix.csv", sep=""), row.names=F)

In [ ]:
# clustering using affinity propagation
x = aba
## compute similarity matrix and run affinity propagation 
## (p defaults to median of similarity)
apres <- apcluster(x, details=TRUE)

## show details of clustering results
show(apres)

## plot clustering result
# plot(apres)

## plot heatmap
# apcluster.heatmap(apres,x)

# ## run affinity propagation with default preference of 10% quantile
# ## of similarities; this should lead to a smaller number of clusters
# ## reuse similarity matrix from previous run
# apres <- apcluster(s=apres@sim, q=0.1)
# show(apres)
# plot(apres, x)

# ## now try the same with RBF kernel
# sim <- expSimMat(x, r=2)
# apres <- apcluster(s=sim, q=0.2)
# show(apres)
# plot(apres, x)


In [ ]:
plot(apres)

In [ ]:
# plot(apres, x)
apcluster::heatmap(apres, x, col=heat.colors(64))

In [ ]:
memb = cutree(hc, k=4)
write.csv(memb, file=paste(DATA_DIR, "nblast_", listName, "_memb.csv", sep=""))

In [ ]:
save.image( paste(DATA_DIR, "nblast_", listName, ".RData", sep=""))

# Evaluation neuron clustering

In [ ]:
library("h5")
f = h5file(file.path(DATASET_DIR,"data.h5"))
neuronIdList = f["nblast/evaluate/neuronIdList"][]
groundTruthPartition = f["nblast/evaluate/groundTruthPartition"][]

flyTableDistanceMatrix = - f["nblast/evaluate/flyTable/meanSimilarityMatrix"][]
zfishTableDistanceMatrix = - f["nblast/evaluate/zfishTable/meanSimilarityMatrix"][]
semanticDistanceMatrix = - f["nblast/evaluate/zfishTable/semantic/meanSimilarityMatrix"][]
h5close(f)

In [ ]:
numType = 15
memb = cutree(hc, k=numType)
write.csv(memb, file=paste(DATA_DIR, "nblast_", listName, "_memb_", numType, "_types.csv", sep=""))